<a href="https://colab.research.google.com/github/viniciusfjacinto/dataops-exercise/blob/main/DataOps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Installing Local MongoDB
# Download link: https://www.mongodb.com/try/download/community


import pandas as pd
import os

#!pip install dotenv
#!pip install pymongo

from pymongo import MongoClient
import dotenv
dotenv.load_dotenv()

# Dataframe Carros
carros_dict = {
    'Carro': ['Onix','Polo','Sandero','Fiesta','City'],
    'Cor': ['Prata','Branco','Prata','Vermelho','Preto'],
    'Montadora': ['Chevrolet','Volkswagen','Renault','Ford','Honda']
}

carros = pd.DataFrame(carros_dict)

#print(carros)

# Dataframe Montadoras

montadoras_dict = {
    'Montadora': ['Chevrolet','Volkswagen','Renault','Ford','Honda'],
    'Pais':['EUA','Alemanha','França','EUA','Japão']
}

montadoras = pd.DataFrame(montadoras_dict)

#print(montadoras)


# MONGOSH SCRIPTS #

# Create collections

#use cars_db

#db.createcollection('carros')
#db.createcollection('montadoras')

# Create Admin User via Mongosh

#use admin

# db.createUser(
#   {
# user: 'admin_dataops',
# pwd: '*******************',
# roles: [
#   { role: "userAdminAnyDatabase", db: "admin"},
#   { role: "userWriteAnyDatabase", db: "admin"}
# ]
# }
# )

#Connection
hostname =  os.environ['mongo_localhost']
port = os.environ['mongo_port']
username = os.environ['mongo_username']
password = os.environ['mongo_pwd']

# Create a MongoClient instance
client = MongoClient(hostname, port, username=username, password=password)

cars_db = client['cars_db']

# Save to MongoDB collections
cars_db['montadoras'].insert_many(montadoras.to_dict('records'))
cars_db['carros'].insert_many(carros.to_dict('records'))

In [ ]:
# MONGO DB Aggregation


# First query - Join carros and montadoras by 'Montadora' field
# [
#     {
#         $lookup: {
#             from: "montadoras",
#             localField: "Montadora",
#             foreignField: "Montadora",
#             as: "joined_data"
#         }
#     },
#     {
#         $unwind: "$joined_data"
#     },
#     {
#         $project: {
#             _id: 1,
#             Carro: 1,
#             Cor: 1,
#             Montadora: 1,
#             Pais: "$joined_data.Pais"  // Include the Pais field from the joined collection
#         }
#     }
# ]


# Second query - Aggregate by 'Pais' field and create new collection

db.carros.aggregate([
  {
    $lookup: {
      from: "montadoras",
      localField: "Montadora",
      foreignField: "Montadora",
      as: "joined_data",
    },
  },
  {
    $unwind: "$joined_data",
  },
  {
    $project: {
      _id: 1,
      Carro: 1,
      Cor: 1,
      Montadora: 1,
      Pais: "$joined_data.Pais", // Include the Pais field from the joined collection
    },
  },
  {
    $group: {
      _id: "$Pais",
      Carros: { $sum: 1 }, // Count the number of documents in each group
    },
  },
  {
    $out: "carros_por_pais", // Output the result to a new collection named 'grouped_result'
  },
])